In [20]:
import redis
from redis.sentinel import Sentinel
import urllib2

In [32]:
# sentinel = Sentinel([('localhost', 40001)], socket_timeout=0.5)
# sentinel.discover_master('masterP_30004')

('127.0.0.1', 30004)

In [2]:
sr = redis.StrictRedis(host='localhost', port=40001)
print sr.sentinel_masters().keys()
p = sr.pubsub()

['masterP_30001', 'masterP_30003', 'masterP_30002', 'masterP_30005', 'masterP_30004', 'masterP_30006']


In [13]:
sr.sentinel_masters()['masterP_30001']['ip']

'127.0.0.1'

In [23]:
def sendListAsPost(url, slist):
    s = ''
    for i in slist: s = s + ',' + i
    s = s[1:]
    data = urllib.urlencode({'update_hash_to_back_map' : s})
    req = urllib2.Request(url, data)
    response = urllib2.urlopen(req)
    result = response.read()
    return result

In [25]:
def getMastersList():
    return [sr.sentinel_masters()[m]['ip'] + ':' + str(sr.sentinel_masters()[m]['port']) for m in sr.sentinel_masters() if not sr.sentinel_masters()[m]['is_sdown']]

def sendMasterList(ml):
    for url in ml:
        ip,port = url.split(':')
        sendListAsPost('https://'+ip+':'+str(int(port) + 20000), ml)

def handleJoin(msg):
    print 'handleJoin():'#, msg['data']
    l = getMastersList()
    print l
    sendMasterList(l)
    
    
    
def handleLeave(msg):
    print 'handleLeave():'#, msg['data']
    l = getMastersList()
    print l
    sendMasterList(l)
    
def handleMessage(message):
    
    t = message['channel']
#     print message['data'], t
    if t == '+sdown':
        handleLeave(message)
    elif t == '-sdown':
        handleJoin(message)
    else:
#         print "Unknown channel message:", message
        t = None

In [19]:
# channels = ['+sdown','-sdown','+odown','-odown','+reboot','-reboot']
channels = ['+sdown','-sdown']
p.psubscribe('*')

for message in p.listen():
    handleMessage(message)

#p.subscribe(None,**{'my-channel': my_handler})

handleLeave():
['127.0.0.1:30001', '127.0.0.1:30005', '127.0.0.1:30004', '127.0.0.1:30006']
handleLeave():
['127.0.0.1:30001', '127.0.0.1:30005', '127.0.0.1:30006']


KeyboardInterrupt: 